In [1]:
using JuMP, Clp, Plots

In [55]:
N = 1
d = collect(60:N:150)
p = (1/size(d,1))*ones(size(d,1));
λ = 0.9
ω = 0.9

0.9

In [3]:
function Q(x,ξ)
    m = Model(Clp.Optimizer)
    set_optimizer_attribute(m, "LogLevel",0)
    @variables(m, 
    begin
        y ≥ 0
        w ≥ 0
    end)

    @constraints(m,
    begin
        y ≤ ξ
        aux, y + w ≤ x
    end)

    @objective(m, Min, -25 * y - 5 * w)
    optimize!(m)
    return objective_value(m), shadow_price(aux)
end

Q (generic function with 1 method)

In [53]:
function cvar(q_list)
    c = Model(Clp.Optimizer)
    set_optimizer_attribute(c, "LogLevel",0)
    @variables(c, 
    begin
        q[1:size(q_list,1)] ≥ 0
    end)

    @constraints(c,
    begin
        sum(q[s] for s = 1:size(q_list,1)) == 1
        aux[s=1:size(q_list,1)], q[s] ≤ (p[s]/(1-λ))
    end)

    @objective(c, Max, sum(q[s]*q_list[s] for s = 1:size(q_list,1)))
    optimize!(c)
    return value.(q)
end 

cvar (generic function with 1 method)

In [56]:
master = Model(Clp.Optimizer)
set_optimizer_attribute(master, "LogLevel",0)
@variable(master,x>=0)
@variable(master,α)
@constraint(master, α >= -9999)
@constraint(master, x<=150)
@objective(master, Min, 10*x + α)
optimize!(master);

iterator = 1
Q̂i_list = []
πi_list = []
x̂i_list = []
LB_list = []
UB_list = []

while true
    LB = objective_value(master)
    x̂i = value(x)
    append!(x̂i_list, x̂i)
    
    slave_Q̄i_list = []
    slave_πi_list = []
    for ξ in d
        q̂, pî = Q(x̂i,ξ)
        append!(slave_Q̄i_list, q̂)
        append!(slave_πi_list, pî)
    end
    
    p_alt = cvar(slave_Q̄i_list)
    
    Q̂i = sum(((1-ω) * p[s] + ω * p_alt[s]) * slave_Q̄i_list[s] for s = 1:size(d,1))
    πi = sum(((1-ω) * p[s] + ω * p_alt[s]) * slave_πi_list[s] for s = 1:size(d,1))


    l_i = x-> Q̂i + πi*(x-x̂i)
    append!(Q̂i_list, Q̂i)
    append!(πi_list, πi)
    
    UB = 10*x̂i + sum(((1-ω) * p[s] + ω * p_alt[s]) * slave_Q̄i_list[s] for s = 1:size(d,1))  
    append!(LB_list, LB)
    append!(UB_list, UB)
        
    if trunc(LB, digits=5) == trunc(UB, digits=5)
        println("**SOLUTION FOUND**")
        println( "Iteration $(iterator): x̂$(iterator): $(x̂i) | Lower Bound: $(LB) & Uper Bound: $(UB)")
        println("===========")
        break
    else
        println( "Iteration $(iterator): x̂$(iterator): $(x̂i) | Lower Bound: $(LB) & Uper Bound: $(UB)")
        println("===========")
    end  
    @constraint(master, α >= l_i(x))
    optimize!(master);
    
    iterator += 1
end

Iteration 1: x̂1: 0.0 | Lower Bound: -9999.0 & Uper Bound: 0.0
Iteration 2: x̂2: 150.0 | Lower Bound: -2250.0000000000055 & Uper Bound: -612.9890109890102
Iteration 3: x̂3: 68.1494505494504 | Lower Bound: -1022.2417582417585 & Uper Bound: -947.5516483516498
Iteration 4: x̂4: 63.99999999999993 | Lower Bound: -960.0000000000011 & Uper Bound: -940.0000000000041
Iteration 5: x̂5: 66.00440528634327 | Lower Bound: -953.986784140971 & Uper Bound: -948.0044052863483
Iteration 6: x̂6: 67.49999999999899 | Lower Bound: -949.500000000004 & Uper Bound: -948.500000000009
Iteration 7: x̂7: 67.00000000000136 | Lower Bound: -949.0000000000066 & Uper Bound: -948.9999999999989
**SOLUTION FOUND**
Iteration 8: x̂8: 66.99999999999758 | Lower Bound: -949.0000000000025 & Uper Bound: -949.0000000000064
